<a href="https://colab.research.google.com/github/AbdulmalikGiwa/Insurance-Claim-Prediction/blob/master/Insuranceclaim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

A predictive model to determine if a building will have an insurance claim during a certain period or not. A few features were provided which will be explored to select the important and best feaures to build the model

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
#importing train set
train = pd.read_csv('/content/train_data.csv')
test = pd.read_csv('/content/test_data.csv')
#Creating an entire column to fill train and test locations with tags to make seperating them easier later on
train['col'] = 'train' 
test['col'] = 'test'
#new dataframe combining train and test
total = pd.concat([train,test], ignore_index=True) 
print(total.shape, train.shape, test.shape)

(10229, 15) (7160, 15) (3069, 14)


# Exploratory Analysis and Data Wrangling

In [3]:
total.isna().sum() #checking for missing values in the dataset

Customer Id              0
YearOfObservation        0
Insured_Period           0
Residential              0
Building_Painted         0
Building_Fenced          0
Garden                  11
Settlement               0
Building Dimension     119
Building_Type            0
Date_of_Occupancy     1236
NumberOfWindows          0
Geo_Code               115
Claim                 3069
col                      0
dtype: int64

## Missing values in 'Garden' column


There are four(4) columns with missing values, In this section I will deal with the values mising in the column of buildings with/without a garden. A K-Nearest Neighbor classifier is used to predict the missing values. The missing values will then be filled by the predicted values. The column is a categorical variable one since there are just two options, and hence the use of a classification algorithm

In [0]:
#LabelEncoder encodes categorical variablees with integers, ML algorithms cannot work with string values
from sklearn.preprocessing import  LabelEncoder
sub_total = total.copy()
le = LabelEncoder() 
#function to get dummies and encode
def tweaks(table) : 
  table = pd.get_dummies(table, columns=['Building_Type'])
  cols = ['Building_Painted', 'Building_Fenced', 'Settlement']
  for i in cols :
    table[i] = le.fit_transform(table[i])

  return table 



#print(sub_total.shape) #shape of new dataframe




In [0]:
colnot = ['Geo_Code', 'Claim', 'col', 'NumberOfWindows', 'Date_of_Occupancy', 'Customer Id',
                        'Building Dimension','Garden']

In [0]:
samp = total.copy()
samp1= tweaks(samp)

In [0]:
# dummy variables creates  new columns for categorical variables and encodes with 1 and 0

class DataCleaner :

  def handle_missing_categorical(dum_gen,col_in_view,col_not_used,model,og_frame):
    #This function will help to sort missing values in columns and return a new dataframe with missing values in
    #the selected column replaced by values predicted by a chosen ML algorithm
    in_ytrain = dum_gen.loc[dum_gen[col_in_view].notnull(), col_in_view]
    in_xtrain = dum_gen.loc[dum_gen[col_in_view].notnull()]
    in_xtrain = in_xtrain.drop(col_not_used, axis=1)
    in_xtest = dum_gen.loc[dum_gen[col_in_view].isnull(), in_xtrain.columns]

    in_ytrain = le.fit_transform(in_ytrain)

    model.fit(in_xtrain, in_ytrain) #fit the model

    pred = model.predict(in_xtest)
    pred= pd.DataFrame(pred)

    samp = dum_gen.loc[dum_gen[col_in_view].isnull()]
    samp2 = samp[['Customer Id']].copy()
    pred.reset_index(drop=True,inplace=True)

    fixed = pd.concat([samp2, pred], axis=1)
    fixed.columns = ['Customer Id', 'Garden']

    og_frame.loc[og_frame[col_in_view].notnull(), col_in_view]

    merged = pd.merge(og_frame, fixed)
    

    og_copy = og_frame.drop(og_frame.loc[og_frame[col_in_view].isnull()].index)
    og_copy[col_in_view] = le.fit_transform(og_copy[col_in_view])
    final_table = pd.concat([og_copy,merged])

    return pred


  def handle_missing_continuous(dum_gen,col_in_view,col_not_used,model,og_frame):
    #This function will help to sort missing values in columns and return a new dataframe with missing values in
    #the selected column replaced by values predicted by a chosen ML algorithm
    in_ytrain = dum_gen.loc[dum_gen[col_in_view].notnull(), col_in_view]
    in_xtrain = dum_gen.loc[dum_gen[col_in_view].notnull()]
    in_xtrain = in_xtrain.drop(col_not_used, axis=1)
    in_xtest = dum_gen.loc[dum_gen[col_in_view].isnull(), in_xtrain.columns]

    in_ytrain = le.fit_transform(in_ytrain)

    model.fit(in_xtrain, in_ytrain) #fit the model

    pred = model.predict(in_xtest)
    pred= pd.DataFrame(pred)

    samp = dum_gen.loc[dum_gen[col_in_view].isnull()]
    samp2 = samp[['Customer Id']].copy()
    pred.reset_index(drop=True,inplace=True)

    fixed = pd.concat([samp2, pred], axis=1)
    fixed.columns = ['Customer Id', col_in_view]

    og_frame.loc[og_frame[col_in_view].notnull(), col_in_view]

    merged = pd.merge(og_frame, fixed)
    

    og_copy = og_frame.drop(og_frame.loc[og_frame[col_in_view].isnull()].index)
    
    final_table = pd.concat([og_copy,merged])

    return pred






  #dummy = tweaks(sub_total)
  #dropping columns not needed
  #dummy.drop(['Claim', 'Geo_Code', 'Building Dimension', 'Date_of_Occupancy', 'NumberOfWindows'], axis=1, inplace=True)

In [0]:
from sklearn.neighbors import KNeighborsClassifier
sampneigh = KNeighborsClassifier(n_neighbors=7)

In [88]:
samp_final = DataCleaner()
samp_final = samp_final.handle_missing(samp1,'Garden',colnot,sampneigh,samp)
print(samp_final.shape)

(11, 1)


In [89]:
samp_final

,0
0,0
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,0
9,0


Customer Id              H1166
YearOfObservation         2013
Insured_Period        0.221918
Residential                  0
Building_Painted             V
Building_Fenced              V
Garden                       V
Settlement                   U
Building Dimension         238
Building_Type                1
Date_of_Occupancy         1980
NumberOfWindows              .
Geo_Code                 86194
Claim                      NaN
col                       test
Name: 9793, dtype: object

In [0]:
#creating these copies to merge original dataframe back with the predicted results
######(backup0.shape, train_backup.shape, backup1.shape, test_backup.shape)

In [0]:
#Creating sub train and test with respect to missing values in Garden column
#sub test and train will be used to run classifier to predict garden

sub_train= dummy.loc[dummy['Garden'].notnull()]
#sub_train= sub_train.reset_index(drop=True)
sub_test= dummy.loc[dummy['Garden'].isnull()]
#sub_test= sub_test.reset_index(drop=True)

In [8]:
sub_train.drop(['Customer Id','col'], axis=1, inplace=True) #drop columns not neede
sub_train['Garden'] = le.fit_transform(sub_train['Garden']) #labe encode garden column
y_train =  sub_train['Garden'].values #y_train will be used to fit model
sub_train.drop(['Garden'], axis=1, inplace=True)
x_train= sub_train # will be used in fitting model
sub_test.drop(['Customer Id', 'col','Garden'],axis=1, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [9]:
print(x_train.shape, sub_train.shape)

(10218, 10) (10218, 10)


In [0]:
from sklearn import preprocessing

x_test = sub_test
X_train= preprocessing.StandardScaler().fit(x_train).transform(x_train) #normalizing data
X_test = preprocessing.StandardScaler().fit(x_test).transform(x_test)

In [0]:
from sklearn.neighbors import KNeighborsClassifier


#This is to select the best no of neighbors for the KNN classifier
def best_neighbors(x,y,samp_num=10) :
  Ks = samp_num
  jacc= np.zeros((Ks-1))

  for n in range(1,Ks) :
      neigh = KNeighborsClassifier(n_neighbors=n).fit(x,y)
      
      jacc[n-1] = neigh.score(x,y)
  return jacc

In [12]:
best_neighbors(x=X_train, y=y_train)

array([1.        , 0.99990213, 0.99990213, 0.99990213, 0.99990213,
       0.99990213, 0.99990213, 0.99990213, 0.99990213])

In [13]:
# fit classifier and make predictions
neigh = KNeighborsClassifier(n_neighbors=1).fit(X_train,y_train)
garden_pred = neigh.predict(X_test)
garden_pred[0:8]

array([1, 1, 1, 1, 1, 1, 1, 1])

In [14]:
garden_pred

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [15]:
y_train[0:8]

array([1, 0, 1, 1, 0, 0, 1, 1])

In [0]:
total['Garden'].fillna('V', inplace = True)

## Missing values in 'Building Dimension'

In [17]:
total.isna().sum()

Customer Id              0
YearOfObservation        0
Insured_Period           0
Residential              0
Building_Painted         0
Building_Fenced          0
Garden                   0
Settlement               0
Building Dimension     119
Building_Type            0
Date_of_Occupancy     1236
NumberOfWindows          0
Geo_Code               115
Claim                 3069
col                      0
dtype: int64

In [18]:
total.nunique()

Customer Id           10229
YearOfObservation         5
Insured_Period          467
Residential               2
Building_Painted          2
Building_Fenced           2
Garden                    2
Settlement                2
Building Dimension     2463
Building_Type             4
Date_of_Occupancy       139
NumberOfWindows          11
Geo_Code               1525
Claim                     2
col                       2
dtype: int64

In [19]:
min(total['Building Dimension'])

1.0

This indicates that the smallest value in building dimension column is 1m^2 which should be impossible, no building is possibly that small, I'll have to do a little bit of research to understand the smallest possible dimension for a building then use this to restructure the column because the problem in this column exceeds just NaN values

In [20]:
total.loc[total['Building Dimension']<30].count()

Customer Id           31
YearOfObservation     31
Insured_Period        31
Residential           31
Building_Painted      31
Building_Fenced       31
Garden                31
Settlement            31
Building Dimension    31
Building_Type         31
Date_of_Occupancy     24
NumberOfWindows       31
Geo_Code              31
Claim                 23
col                   31
dtype: int64

From some research I did through sources online, the minimum dimension for a building is expected to fall within the range of 30m^2 to 37m^2, for this reason I'll take 30 as the minimum acceptable dimension therefore every row with values of Building Dimension being lesser than 30  will be replaced by NaN. After sorting the irregularity in the dimension, A Linear Regression algorithm will be used to predict all the unknown values 

In [0]:
bd= total.copy()
bd.loc[bd['Building Dimension']<30, 'Building Dimension']= np.nan #replaces all values less than 30 with Nan


In [34]:
bd.isna().sum()

Customer Id              0
YearOfObservation        0
Insured_Period           0
Residential              0
Building_Painted         0
Building_Fenced          0
Garden                   0
Settlement               0
Building Dimension     150
Building_Type            0
Date_of_Occupancy     1236
NumberOfWindows          0
Geo_Code               115
Claim                 3069
col                      0
dtype: int64

In [0]:
bd= tweaks(bd) #tweaks function defined in a cell earlier above
y_bdtrain = bd.loc[bd['Building Dimension'].notnull(), 'Building Dimension']

x_bdtrain = bd.loc[bd['Building Dimension'].notnull()]

x_bdtrain = x_bdtrain.drop(['Geo_Code', 'Claim', 'col', 'NumberOfWindows', 'Date_of_Occupancy', 'Customer Id',
                       'Building Dimension'], axis=1)


x_bdtest = bd.loc[bd['Building Dimension'].isnull(), x_bdtrain.columns]

In [36]:
print(x_bdtest.shape)

(150, 11)


In [0]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()

lr.fit(x_bdtrain, y_bdtrain) #fit the model

bd_pred = lr.predict(x_bdtest)

In [0]:
bd_pred = pd.DataFrame(data=bd_pred)

In [0]:
bd4test = bd.loc[bd['Building Dimension'].isnull()]
bdtest = bd4test[['Customer Id']].copy()
bd_fixed = pd.concat([bdtest, bd_pred], axis=1)

In [49]:
print(bdtest.shape, bd_pred.shape)

(150, 1) (150, 1)


In [56]:
bd_fixed = pd.concat([bd_pred, bdtest2], axis=1)
bd_fixed.head()

,0,Customer Id
0,1676.0,H4555
1,1676.0,H6126
2,1848.0,H4585
3,1852.0,H4586
4,1780.0,H6756


0              148
Customer Id    148
dtype: int64

In [0]:
bdtest2 = bdtest.reset_index(drop=True)